In [2]:
#1. import necessary libraries and modules

#2. upload all necessary data

#3. define X and y. y - our target. X- predictors. We have X but how we represent y? -  Vlad say that for 
#   now we do not devide our dataset into two this parts, but I suppose you should ask Markiyan about
#   how we can measure the efficiency of the model later on

#4. devide data into train and test using following link
    #use https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

#5. train 6 models:
    # 1. LogisticRegression
    # 2. Support Vector Machines
    # 3. Stochastic Gradient Descent
    # 4. Random Forest
    # 5. Nearest Neighbors Classification
    # 6. Bayesian Regression
    
#6. save them

#7 Calculate confusion matrices for each model

#8 Choose the top 3 models. Motivate your decision on how did you define “top” models?

#9 Tune hyperparameters for the top 3 models you choose

#10. move to the next file - 5_evaluate_models